In [1]:
import requests
import json
import os

In [2]:
default_navitia_url = "https://api.navitia.io/v1"
default_navitia_api_key = ""

navitia_url = os.getenv('navitia_url', default_navitia_url)
navitia_api_key = os.getenv('navitia_api_key', default_navitia_api_key)

In [3]:
network_id_in_navitia = "network:0:642"

In [4]:
operator_mapping_list = {
    "MARNE ET MORIN" : "Marne et Morin",
    "DARCHE GROS": "Transdev Darche-Gros",
    "CIF": "Keolis CIF",
    "SAVAC": "SAVAC",
    "Transdev Ile-de-France Rambouillet": "Transdev Île-de-France Rambouillet",
    "Transdev Ile-de-France St-Fargeau-Ponthierry" : "Transdev Île-de-France Saint-Fargeau-Ponthierry"
}

network_mapping_list = {
    "Pays Fertois" : "Pays Fertois",
    "Pays de l'Ourcq" : "Pays de l'Ourcq",
    "Pays Créçois" : "Pays Créçois",
    "CIF": "Les Courriers de l'Île-de-France",
    'Goëlys' :'Goëlys',
    "SAVAC": "SAVAC",
    "Réseau du Canton de Perthes" :"Canton de Perthes en Gâtinais",
    "Rambouillet Interurbain" : "Réseau Interurbain de Rambouillet"
}

In [5]:
lines_url = "{}/coverage/fr-idf/networks/{}/lines/".format(navitia_url, network_id_in_navitia) 
#TODO, il peut y avoir plusieurs pages !!
lines_call = requests.get(lines_url, headers={'Authorization': navitia_api_key})
lines_result_json = lines_call.json()
if not 'lines' in lines_result_json:
    print('KO navitia')
    all_lines = []
else :
    all_lines=lines_result_json['lines']
    

In [6]:
text_result = ""

for a_line in all_lines:
    text_result += """
# Ligne {}    
    """.format(a_line['code'])
    
    line_interop_codes = {elem['type']: elem['value'] for elem in a_line['codes']}
    if 'osm_line_id' in line_interop_codes:
        text_result += """
http://openstreetmap.org/relation/{}
        """.format(line_interop_codes['osm_line_id'].split(':')[-1])
        continue
    
    # on récupère la bonne valeur pour le tag operator
    company_url = "{}/coverage/fr-idf/lines/{}/companies".format(navitia_url, a_line['id'])
    company_call = requests.get(company_url, headers={'Authorization': navitia_api_key})
    company = company_call.json()['companies'][0]['name']
    operator = operator_mapping_list[company]
    
    # on essaye de deviner la destination
    if len(a_line['routes']) != 2:
        text_result += """
Attention, la ligne a {} trajet(s)        
        """.format(len(a_line['routes']))
        dest = "TODO"
    else :
        dest = a_line['routes'][1]['direction']['name']
    
    #on crée le template pour la ligne à créer
    init_template = """
{{{{User:Singing-Poppy/Template:InitPTRelations
|ref = {ref}
|from = {from_}
|to = {to}
|operator = {operator}
|network = {network}
|pt_mode = bus
}}}}
    """.format(
        ref=a_line['code'],
        from_=a_line['routes'][0]['direction']['name'],
        to=dest, 
        operator=operator,
        network=network_mapping_list[a_line['network']['name']],
    )
    
    text_result += """
Template à ajouter sur le wiki si la ligne n'existe pas encore dans OSM :
```{}
```
    """.format(init_template)
    
    #on crée le template pour la ligne une fois créée
    wiki_line = """
|-
|style="background-color:#{color}; text-align: center; color: #000000" rowspan="{nb_routes}"| {ref} 
|colspan="3"| {{{{browseLine|0||ref={ref}|style=paris}}}} '''TODO''' ||
|-""".format(
        ref=a_line['code'],
        color=a_line['color'],
        nb_routes=len(a_line['routes']) +1,
    )
    for a_route in a_line['routes']:
        wiki_line+= """
|| {{Relation|0| TODO |tools=no}} ||  || {{State Route|r=NA|h=NA}} ||
|- """
        
    text_result += """
Template à ajouter sur le wiki une fois la ligne créée :
```{}
```
    """.format(wiki_line)
    
print(text_result)
    


# Ligne 01    
    
Attention, la ligne a 1 trajet(s)        
        
Template à ajouter sur le wiki si la ligne n'existe pas encore dans OSM :
```
{{User:Singing-Poppy/Template:InitPTRelations
|ref = 01
|from = Gare de Ponthierry Pringy (Saint-Fargeau-Ponthierry)
|to = TODO
|operator = Transdev Île-de-France Saint-Fargeau-Ponthierry
|network = Canton de Perthes en Gâtinais
|pt_mode = bus
}}
    
```
    
Template à ajouter sur le wiki une fois la ligne créée :
```
|-
|style="background-color:#799C13; text-align: center; color: #000000" rowspan="2"| 01 
|colspan="3"| {{browseLine|0||ref=01|style=paris}} '''TODO''' ||
|-
|| {{Relation|0| TODO |tools=no}} ||  || {{State Route|r=NA|h=NA}} ||
|- 
```
    
# Ligne 09    
    
Template à ajouter sur le wiki si la ligne n'existe pas encore dans OSM :
```
{{User:Singing-Poppy/Template:InitPTRelations
|ref = 09
|from = Gare de Melun (Melun)
|to = Cornebiche (Arbonne-la-Forêt)
|operator = Transdev Île-de-France Saint-Fargeau-Ponthierry
|networ

In [7]:
with open('wiki.md', 'w') as out_md_file:
    out_md_file.write(text_result)